# Homework 05

In [37]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import statistics
import pickle
import pandas as pd

## Let's build the graph

We had to build the graph properly and provide basic informations such as:

   * If it is direct or not
   * The number of nodes
   * The number of edges
   * The average node degree. Is the graph dense?
   

We used a package called **networkX** that has been used in the labs with prof Ioannis.
   

In [15]:
G = nx.read_edgelist("wiki-topcats-reduced.txt", create_using=nx.DiGraph())

We used the built in function of networkX to get all these basics informations. We have **461193** nodes and an impressive **2645247** number of edges.

In [13]:
nx.info(G)

'Name: \nType: DiGraph\nNumber of nodes: 461193\nNumber of edges: 2645247\nAverage in degree:   5.7357\nAverage out degree:   5.7357'

The graph that we deal with is directed graph according to the *"wiki-topcats-reduced.txt"*. In this text file, every row is an edge, the two elements are the nodes (source and destination). Some pairs of nodes are shown twice while the source and destination are opposite. Thus,  we decide that this graph is a **directed graph**.

In [5]:
G.is_directed()

True

In [5]:
G.number_of_nodes()

461193

In [6]:
G.number_of_edges()

2645247

Since this graph is directed , the **average degree** is the number of edges divided by the number nodes: 

$$Average \ degree = \frac{the \ number \ of \ edges}{the \ number \ of \ nodes}$$

To retrieve this information we just had to use the followinf networkX functions:

In [7]:
G.number_of_edges() / G.number_of_nodes()


5.735661642739591

The last information we had to provide about this graph is its **density**. The density of a directed graph is defined as:


$$Density = \frac{the \ number \ of \ edges}{the \ number \ of \ nodes(the \ number \ of \ nodes - 1)}$$

In [8]:
density = G.number_of_edges() / (G.number_of_nodes()* (G.number_of_nodes()- 1))
print(density)
print(nx.density(G))

1.2436602635647606e-05
1.2436602635647606e-05


We can clearly say that this graph is **not dense**.

In [9]:
nx.info(G)

'Name: \nType: DiGraph\nNumber of nodes: 461193\nNumber of edges: 2645247\nAverage in degree:   5.7357\nAverage out degree:   5.7357'

We called again the info function just to double-check our results. The **average degree**, number of nodes and edges are the same as the ones we obtained before.

***
# RQ2
***



### Obtain a _block-ranking_, where the blocks are represented by the categories
   
   - How to order categories?
       1. Pick first category $C_0$
       2. compute distance between nodes in the category($C_0$) and nodes of other categories($C_i$)
       
       $$distance(C_0, C_i)  = median(ShortestPath(C_0, C_i))$$
              
           - $ShortestPath(C_0, C_i)$ is the set of all the possible shortest paths between the nodes of $C_0$ and $C_i$
           - length of a path is given by the sum of the weights of the edges
           
### Once you obtain the _block-ranking_ vector, we sort the nodes in each category.
    
   - How to sort the nodes?
       1. Compute subgraph induced by $C_0$. For each node compute the sum of the weigths of the in‑edges.
          $$score_{article_i} = \sum_{i\in{in-edges}} w_i$$ 
          
       2. Extend the graph to the nodes that belong to $C_1$. Thus, for each article in $C_1$ compute the score as before. 
          **Note** that the in‑edges coming from the previous category,$C_0$, have as weights the score of the node that sends the edge.
    
       3. Repeat Step2 up to the last category of the ranking. 
       
  

## Import the text files

We start importing the files we need. We first import *wiki-topcats-reduced.txt* that contains nodes(source and destinations) and edges.

In [10]:
# Get the numbe of nodes in this file
all_nodes = []
with open("wiki-topcats-reduced.txt") as file:
    for edge in file:
        all_nodes.extend(edge.split())
all_nodes = set(all_nodes)

# the number of nodes
len(all_nodes)

461193

In [11]:
edges = []
with open("wiki-topcats-reduced.txt") as file:
    for edge in file:
        edges.append(edge.split())
        
# the number of edges
len(edges)

2645247

We then computed,to explore properly the data-set, the *distinct* number of sources and destinations in our file.

In [16]:
#We want to know how many different sources we have in the above file
sources = []
for nodes in edges:
    sources.append(nodes[0])
sources = set(sources)
len(sources)

428957

In [17]:
#We want to know how many different destinations we have in the above file
destinations = []
for nodes in edges:
    destinations.append(nodes[1])
destinations = set(destinations)
destinations = sorted(destinations, key=int)
len(destinations)

352518

We then proceeded importing the *wiki-topcats-categories.txt* file that contains all the different categories and all the articles belonging to the. We consider only categories having more than **3500 articles**.

In [18]:
# Create a categories dictionary
categories0 = {}
with open("wiki-topcats-categories.txt") as file:
    for category in file:
        category = category.strip("Category:").replace("\n", "").replace(";", "").split()
        categories0[category[0]] = category[1:]

In [19]:
# Extract only categories which have 3500 articles
categories = {}
for k,v in categories0.items():
    if len(v) > 3500:
        categories[k] = v
        
# the number of categories that we deal with
len(categories.keys())

35

We want to consider only articles present in the reduced file. We took the intersection in order to cut out the nodes of which we had no information about. We print out the result to show the numerosity of every category.

In [20]:
for cate,nodes in categories.items():
    
    # Get only intersection between nodes in category and all nodes
    categories[cate] = list(set(nodes).intersection(all_nodes))

num_nodes_reduced = {}
for cate,nodes in categories.items():
    num_nodes_reduced[cate] = len(nodes)

# this is the categories and the number of nodes that we will work with
num_nodes_reduced

{'English_footballers': 7538,
 'The_Football_League_players': 7814,
 'Association_football_forwards': 5097,
 'Association_football_goalkeepers': 3737,
 'Association_football_midfielders': 5827,
 'Association_football_defenders': 4588,
 'Living_people': 348300,
 'Year_of_birth_unknown': 2536,
 'Harvard_University_alumni': 5549,
 'Major_League_Baseball_pitchers': 5192,
 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies': 6491,
 'Indian_films': 5568,
 'Year_of_death_missing': 4122,
 'English_cricketers': 3275,
 'Year_of_birth_missing_(living_people)': 28498,
 'Rivers_of_Romania': 7729,
 'Main_Belt_asteroids': 11660,
 'Asteroids_named_for_people': 4895,
 'English-language_albums': 4760,
 'English_television_actors': 3362,
 'British_films': 4422,
 'English-language_films': 22463,
 'American_films': 15159,
 'Fellows_of_the_Royal_Society': 3446,
 'People_from_New_York_City': 4614,
 'American_Jews': 3411,
 'American_television_actors': 11531,
 'American_film_actors': 13865,


As explained in the homework text, we had to possibility to choose the category in input. We opted to choose the less populated one,**'Year_of_birth_unknown'**, in order to make our computations less heavier. 

In [21]:
c0 = 'Year_of_birth_unknown'
all_ci = list(categories.keys())
all_ci.remove(c0) #we remove c0 form the list of all the categories in which we have to compute the distance

In [22]:
nodes_c0 = categories[c0] #this are the nodes corresponding to c0 category
nodes_c0.sort(key=int)
len(nodes_c0) 

2536

We defined two functions in order to be able to use picke files. We used that to save the ranked result(our output) in an external file in order to be able to load it up when needed without making all the computations again.

In [2]:
def write_file_to_pickle(file, content):
    with open(file, "wb") as f:
        pickle.dump(content, f)
        f.close()

def read_file_from_pickle(file):
    file_content = {}
    with open(file, "rb") as f:
        file_content = pickle.load(f)
        f.close()
    
    return file_content

In order to explore our graph, we decided to use Breadth-first search (BFS). It is an algorithm for traversing or searching tree or graph data structures. It starts at the tree root (or some arbitrary node of a graph) and explores all of the neighbor nodes at the present depth prior to moving on to the nodes at the next depth level. 

This algorithm has mainly two features(the reasons why we opted to choose this one):

*     it uses a queue (First In First Out) instead of a stack (Last In First Out) and
*     it checks whether a vertex has been discovered before enqueueing the vertex rather than delaying this check until the vertex is dequeued from the queue.

In [54]:
def bfs(Graph, source, destination):
    queue = [[source]]  # we take the set of the sources
    visited = set()  #we create a set in which we will store all the visited nodes
    
    if not nx.has_path(Graph, source, destination): #If there is no path between source and destination we return nan
        return float("nan")
    
    while queue:   #While we have some nodes to explore
        # Gets the first path in the queue
        path = queue.pop(0)

        # Gets the last node in the path
        vertex = path[-1]

        # Checks if we got to the end
        if vertex == destination:
            return len(path)
        # We check if the current node is already in the visited nodes set in order not to recheck it
        elif vertex not in visited:
            neighbours = list(Graph.neighbors(vertex))
            # enumerate all adjacent nodes, construct a new path and push it into the queue
            for current_neighbour in neighbours:
                new_path = list(path)
                new_path.append(current_neighbour)
                queue.append(new_path)
                
            # Mark the vertex as visited
            visited.add(vertex)

In [ ]:
block_ranking = {} #This will contain the categories ranked.

for ci in all_ci:
    # pick one category
    nodes_ci = categories[ci]
    
    #make a subgraph
    sub_nodes = nodes_c0 + nodes_ci
    H = G.subgraph(sub_nodes)
    
    # a list to input the shortest path
    length = []

    for source in nodes_c0:
        for destination in nodes_ci:
            # compute shortest path
            length.append(bfs(H, source, destination))
    
    # remove nan value
    length = [ x for x in length if str(x) != "nan"]
    # get a median in list and append it to the dictionary
    block_ranking[ci] = statistics.median(length)
    
# sort by value
block_ranking = sorted(block_ranking.items(), key = lambda x: x[1])

# save this dictionary to avoid costing much time again 
write_file_to_pickle("block_ranking", block_ranking)

After having read from the pickle file this is the output of our categories ranked by the median of the shortest path.

In [121]:
block_ranking = read_file_from_pickle("block_ranking")
block_ranking

[('Major_League_Baseball_pitchers', 2),
 ('Debut_albums', 2),
 ('Year_of_death_missing', 3.0),
 ('Year_of_birth_missing_(living_people)', 3),
 ('Year_of_birth_missing', 3),
 ('Place_of_birth_missing_(living_people)', 3),
 ('Asteroids_named_for_people', 4.5),
 ('Main_Belt_asteroids', 5),
 ('British_films', 5),
 ('Association_football_goalkeepers', 7),
 ('Article_Feedback_Pilot', 7),
 ('Rivers_of_Romania', 7.0),
 ('English_footballers', 8),
 ('The_Football_League_players', 8.0),
 ('Association_football_forwards', 8.0),
 ('Association_football_midfielders', 8.0),
 ('Association_football_defenders', 8.0),
 ('English_cricketers', 8.0),
 ('English_television_actors', 8.0),
 ('Fellows_of_the_Royal_Society', 8.0),
 ('American_Jews', 8),
 ('American_television_actors', 8),
 ('American_film_actors', 8.0),
 ('American_military_personnel_of_World_War_II', 8),
 ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 8),
 ('Living_people', 8.0),
 ('Indian_films', 9),
 ('English-langu

We load back the *sorted_notes* which contains the nodes sorted.

In [ ]:
categories_block_ranking = [x[0] for x in block_ranking ]
categories_block_ranking.insert(0, c0)
len(categories_block_ranking)

In [ ]:
# make nested dictionary(key1 : category, key2(value1) : node, value2 :  weight)
nodes_rank = {}

for idx, category in enumerate(categories_block_ranking):
    
    # nested dictionary like 
    # nodes_rank { category { node : weight } }
    
    nodes_rank[category] = {}
    nodes = categories[category]
    
    # This is only first category
    if idx == 0:
        
        # make subgraph
        H = G.subgraph(nodes)
        
        for node in nodes:
            # first set 0 as initial weight
            nodes_rank[category][node]  = 0
            # find neighbors
            neighbors = list(H.neighbors(node))
            
            for neighbor in neighbors:
                # assign weight 
                # check whether the edge is coming from neighbor to node or not
                if nx.has_path(H, neighbor, node):
                    nodes_rank[category][node] += 1
        
    else:
        # make subgraph of current category and previous category
        sub_nodes = nodes + previous_nodes
        H = G.subgraph(sub_nodes)
        
        for node in nodes:
            # first set 0 as initial weight
            nodes_rank[category][node]  = 0
            # find neighbors
            neighbors = list(H.neighbors(node))
            
            for neighbor in neighbors:
                # if the edge comes from PREVIOUS category, add the weight of the node which comes from
                if neighbor in previous_nodes and nx.has_path(H, neighbor, node):
                    nodes_rank[category][node] += nodes_rank[previous_category][neighbor]
                    continue
                # if the edge comes from CURRENT category
                elif nx.has_path(H, neighbor, node):
                    nodes_rank[category][node] += 1
                    
    # save nodes and category as previous data         
    previous_nodes = nodes
    previous_category = category

# sort values in dictionary
for i in range(len(nodes_rank)):
    category = categories_block_ranking[i]
    nodes_rank[category] = sorted(nodes_rank[category].items(), key = lambda x: x[1], reverse=True)    

# save as a pickle file
write_file_to_pickle("sorted_nodes", nodes_rank)

In [122]:
sorted_nodes = read_file_from_pickle("sorted_nodes")

We import also the third file contatining all the names related to the categories.

In [123]:
listanomi=[]
with open('wiki-topcats-page-names.txt', 'r') as f:
    for line in f.readlines():
        l = line.split(' ',1)
        listanomi.append(l[1])
lista1 = [w.replace('\n', ' ') for w in listanomi]   #in this list we have all the names cleaned

In [124]:
lst=[]
for i in block_ranking:
    lista=[]
    m=0
    for s in sorted_nodes[str(i[0])]: #We select just first ten observation to make a little output of our results
        if m<10:
            lista.append(s[0])
            m=m+1
    lst.append(lista)
for i in lst:
    for m in range(0,len(i)): #With this we retrieve the names of the articles corresponding to the indexes
        value=int(i[m])-1
        i[m]=lista1[value] 
#And finally...we create a DataFrame        
data_tuples = list(zip(lst[0],lst[1],lst[2],lst[3],lst[4],lst[5],lst[6],lst[7],lst[8],lst[9],lst[10],lst[11],lst[12],lst[13],lst[14],lst[15],lst[16],lst[17],lst[18],lst[19],lst[20],lst[21],lst[22],lst[23],lst[24],lst[25],lst[26],lst[27],lst[28],lst[29],lst[30],lst[31],lst[32],lst[33]))
df=pd.DataFrame(data_tuples,columns=['Major_League_Baseball_pitchers','Debut_albums','Year_of_death_missing','Year_of_birth_missing_(living_people)','Year_of_birth_missing','Place_of_birth_missing_(living_people)','Asteroids_named_for_people','Main_Belt_asteroids','British_films','Association_football_goalkeepers', 'Article_Feedback_Pilot', 'Rivers_of_Romania',  'English_footballers', 'The_Football_League_players',  'Association_football_forwards',  'Association_football_midfielders', 'Association_football_defenders',  'English_cricketers', 'English_television_actors', 'Fellows_of_the_Royal_Society', 'American_Jews', 'American_television_actors', 'American_film_actors',  'American_military_personnel_of_World_War_II', 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 'Living_people', 'Indian_films', 'English-language_albums', 'Harvard_University_alumni', 'People_from_New_York_City', 'English-language_films',
 'American_films', 'Black-and-white_films', 'Windows_games'])

#It has 34 columns as we have excluded the input category.

df

,Major_League_Baseball_pitchers,Debut_albums,Year_of_death_missing,Year_of_birth_missing_(living_people),Year_of_birth_missing,Place_of_birth_missing_(living_people),Asteroids_named_for_people,Main_Belt_asteroids,British_films,Association_football_goalkeepers,...,Members_of_the_United_Kingdom_Parliament_for_English_constituencies,Living_people,Indian_films,English-language_albums,Harvard_University_alumni,People_from_New_York_City,English-language_films,American_films,Black-and-white_films,Windows_games
0,Luis Aparicio,The Emperor of the Bathroom,Don Rowlands,Steve Yeowell,Geoffrey Clarkson,Gordon Haynes,3353 Jarvis,3351 Smith,Bud Flanagan,201011 Blackburn Rovers F.C. season,...,Nic Dakin,Life peer,Patton (film),Ballad of Easy Rider,Chris Leslie (folk musician),Critique of Pure Reason,Eleanor and Franklin: The White House Years,List of The Nostalgia Critic episodes,A Night at the Opera (film),Limbo of the Lost
1,Tom Seaver,It Was Written,Thomas Engel,Margaret Way,Jane Donnelly,John McKeown (rugby league),3352 McAuliffe,3355 Onizuka,Q (James Bond),Peter Clarke (footballer),...,Julian Ridsdale,James Callaghan,Munna Bhai M.B.B.S.,List of Basement Tapes songs,Powder Alarm,Karl Earl Mundt,Terror in the Aisles,Butch Cassidy and the Sundance Kid,Small Soldiers,Sam & Max Beyond Time and Space
2,Philadelphia Phillies,Ol' Dirty Bastard,Charlie Davey (footballer),Lisa Connor,James Gaines,Geoff Wriglesworth,3351 Smith,3353 Jarvis,The Best Man (1964 film),Kak,...,Stockton South (UK Parliament constituency),Order of succession,Moojrim,Art Garfunkel,Augustus Lowell,William Warren Barbour,William S. Cowles,Haskell Wexler,Monkey Business (1931 film),Penumbra: Requiem
3,Pete Rose,The Rest Is History,Harry Clarke (footballer),Mills & Boon,Don Rowlands,William Banks (rugby league),3350 Scobee,3352 McAuliffe,For Your Eyes Only (film),Joe Kinnear,...,Harold Wilson,John Major,Sanjay Leela Bhansali,Harvest Moon (album),Edgard Varse,Herbert Hoover,Bruce Greenwood,A Night at the Opera (film),Animal Crackers (film),Dragon Age: Origins
4,Earned run average,Verizon Wireless Amphitheater (Selma),Kerry Ashby,Brian Tarquin,Thomas Engel,George Parsons (rugby),3356 Resnik,3356 Resnik,Liz Fraser,Gianluca Zanetti,...,Secretary of State for Education,Martin Stevens,Zor (film),I Don't Want to Spoil the Party,Presidency of Thomas Jefferson,United States Ambassador to the United Nations,List of Sundance Film Festival selections,Frank Marshall (film producer),James Rebhorn,Dungeon Runners
5,Nolan Ryan,Liquid Swords,Jack L. Collins,Catherine Spencer,Kerry Ashby,Israa Abdel Fattah,3355 Onizuka,3350 Scobee,Johnny English,Marko Panteli,...,Labour Government 19451951,"John Boyd-Carpenter, Baron Boyd-Carpenter",Rekha,It Was Written,Harry Markowitz,William Maclay (politician),A Night at the Opera (film),The Last House on the Left (2009 film),The Bell Boy,Aquaria (video game)
6,Derek Lowe,Cliff Burton,Murray Ashby,Glenn Fabry,Murray Ashby,Heather Armitage,2383 Bradley,2383 Bradley,Licence to Kill,Ronnie Wallwork,...,"Victor Cavendish, 9th Duke of Devonshire",Edward Leigh,Jayaraj,Across the Universe,Philosophy of logic,Stuart Rothenberg,Gustave Reininger,Golden Globe Award for Best Screenplay,Virginia Fox,Atom Zombie Smasher
7,Steve Carlton,So Fresh: The Hits of Autumn 2007,Bruce Culpan,Sara Wood (novelist),Ian Watson (rugby league),Jeff Fishback,9531 Jean-Luc,9621 Michaelpalin,James Bond Theme,Lee Sharpe,...,Aubrey Jones,Howard Vincent,Udhayanidhi Stalin,Disturbia (song),Guy Lowell,Howl,Casablanca (film),Meg Tilly,Norman Z. McLeod,SimCity 2000
8,George Steinbrenner,Echo (Leona Lewis album),Dmitry Golovanov,Cathy Williams,Bruce Culpan,Lau Kwok Kin,9620 Ericidle,9531 Jean-Luc,Live and Let Die (film),Patrick Vieira,...,Holborn and St Pancras South (UK Parliament co...,James Barr (politician),3 Idiots,Revolver (album),Philosophical progress,EXCOMM,List of Academy Award-winning films,Walt Disney Pictures,The Cat's Meow,Half-Life (video game)
9,Frank Francisco,Right Round,Donald A